---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [23]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [24]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [25]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni_town = pd.read_table('university_towns.txt', header=None)
    
    def splitter(row, table, param):
        if param['state'] == '' or row[0].find('edit') > 0:
            param['state'] = row[0].split('[')[0]
        else:
            town = row[0].split('(')[0].strip()
            table.append([param['state'], town])
        
    param = {'state': '' }
    table = []
    uni_town.apply(lambda row: splitter(row, table, param), axis=1)
    
    uni_town = pd.DataFrame(table, columns=["State", "RegionName"])
    
    '''data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])'''

    return uni_town
#get_list_of_university_towns()

In [26]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=220, header = None)
    gdp = pd.DataFrame(gdp)
    gdp = gdp[[4, 6]]
    gdp.columns = ['Quarters','GDP']
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    
    
    rec_start = []
    
    for i in range(len(gdp) - 2):
        if gdp.iloc[i]["GDP"] > gdp.iloc[i+1]["GDP"] and gdp.iloc[i+1]["GDP"] > gdp.iloc[i+2]["GDP"]:
            rec_start.append(gdp.iloc[i+1]["Quarters"])
            #rec_start = (gdp.iloc[i]["Quarters"]) #Returns last value of recession
    
    
    
    return rec_start[0] #Return first beggining of recession

#get_recession_start()

In [27]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
     
    gdp = pd.read_excel('gdplev.xls', skiprows=220, header = None)
    gdp = pd.DataFrame(gdp)
    gdp = gdp[[4, 6]]
    gdp.columns = ['Quarters','GDP']
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    
    
    start_index = gdp[gdp['Quarters'] == get_recession_start()].index.tolist()[0]
    
    rec_end = []
    
    for i in range(start_index, len(gdp) - 2):
        if gdp.iloc[i]["GDP"] < gdp.iloc[i+1]["GDP"] and gdp.iloc[i+1]["GDP"] < gdp.iloc[i+2]["GDP"]:
            rec_end.append(gdp.iloc[i+2]["Quarters"])
            #rec_end = gdp.iloc[i+2]["Quarters"] #Returns end value of recession 

    
    
    
    return rec_end[0] #returns the first value of the list

#get_recession_end()

In [28]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    
    gdp = pd.read_excel('gdplev.xls', skiprows=220, header = None)
    gdp = pd.DataFrame(gdp)
    gdp = gdp[[4, 6]]
    gdp.columns = ['Quarters','GDP']
    gdp['GDP'] = pd.to_numeric(gdp['GDP'])
    
    
    start_index = gdp[gdp['Quarters'] == get_recession_start()].index.tolist()[0]
    end_index = gdp[gdp['Quarters'] == get_recession_end()].index.tolist()[0]
    
    
    recession = gdp.iloc[start_index:end_index + 1]
    
    bottom_val = recession['GDP'].min()
    
    
    bottom = recession.loc[recession['GDP'] == bottom_val, 'Quarters'].iloc[0]
    return bottom

#get_recession_bottom()

In [29]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

    house_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_data = house_data.drop(house_data.columns[[0] + list(range(3,51))], axis=1)

    house = pd.DataFrame(house_data[['State', 'RegionName']])
    
    for year in range(2000, 2016):
        house[str(year) + 'q1'] = house_data[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
        house[str(year) + 'q2'] = house_data[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
        house[str(year) + 'q3'] = house_data[[str(year) + '-07', str(year) + '-08', str(year) + '-09']].mean(axis = 1)
        house[str(year) + 'q4'] = house_data[[str(year) + '-10', str(year) + '-11', str(year) + '-12']].mean(axis = 1)
    year = 2016
    house[str(year) + 'q1'] = house_data[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
    house[str(year) + 'q2'] = house_data[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
    house[str(year) + 'q3'] = house_data[[str(year) + '-07', str(year) + '-08']].mean(axis = 1)

    house['State'] = [states[state] for state in house['State']]
    house = house.set_index(['State', 'RegionName'])
   
    return house
#convert_housing_data_to_quarters()


In [30]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    price = convert_housing_data_to_quarters()
    
    unitown = get_list_of_university_towns()
    start = get_recession_start()
    bottom = get_recession_bottom()
    house = convert_housing_data_to_quarters()
    
    before_start = house.columns[house.columns.get_loc(start) -1]


    price = house.loc[:,before_start:bottom]
    
    price = price.reset_index()
    
    
    price ['Ratio'] = price[before_start]/price[bottom]
    
    
    unitown ['Unitown'] = True
    
    df = pd.merge(price, unitown, how='left', on=['State', 'RegionName'])
    
    df ['Unitown'] = df['Unitown'].fillna(False)
    
    
    ut = df[df['Unitown'] == True]
    nut = df[df['Unitown'] == False]
    
    t, p = ttest_ind(ut['Ratio'].dropna(), nut['Ratio'].dropna())
    
    different = True if p < 0.01 else False
    
    better = "university town" if ut['Ratio'].mean() < nut['Ratio'].mean() else "non-university town"
    
    
    
    return (different, p, better)



#run_ttest()